In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

fatal: destination path 'facial_expressions' already exists and is not an empty directory.


In [2]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]


In [3]:
emotion_list = list(data.keys())
len(emotion_list), emotion_list

(8,
 ['anger',
  'surprise',
  'disgust',
  'fear',
  'neutral',
  'happiness',
  'sadness',
  'contempt'])

In [4]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')


FileExistsError: ignored

In [ ]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training', emotion))
  os.mkdir(os.path.join('master_data/training', emotion))
  os.mkdir('master_data/testing')

In [ ]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    destination = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, destination)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    destination = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, destination)
  

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [6]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3, 3), activation='relu', input_shape = (100, 100,3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3, 3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.001), loss='categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (100, 100),
    class_mode = 'categorical',
    batch_size = 128
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (100, 100),
    class_mode = 'categorical',
    batch_size = 128
)




Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [8]:
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

In [9]:
model.fit_generator(train_generator,
                    epochs = 100,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/100
86/86 [==============================] - 20s 195ms/step - loss: 0.8728 - acc: 0.6721 - val_loss: 1.4215 - val_acc: 0.6295
Epoch 2/100
86/86 [==============================] - 16s 189ms/step - loss: 0.5705 - acc: 0.8061 - val_loss: 1.3960 - val_acc: 0.6565
Epoch 3/100
86/86 [==============================] - 16s 191ms/step - loss: 0.4980 - acc: 0.8306 - val_loss: 1.4805 - val_acc: 0.6736
Epoch 4/100
86/86 [==============================] - 17s 193ms/step - loss: 0.4387 - acc: 0.8484 - val_loss: 1.7188 - val_acc: 0.6823
Epoch 5/100
86/86 [==============================] - 16s 190ms/step - loss: 0.3969 - acc: 0.8635 - val_loss: 1.7650 - val_acc: 0.6878
Epoch 6/100
86/86 [==============================] - 16s 191ms/step - loss: 0.3538 - acc: 0.8771 - val_loss: 1.8835 - val_acc: 0.6893
Epoch 7/100
86/86 [==============================] - 16s 189ms/step - loss: 0.3274 - acc: 0.8808 - val_loss: 1.7703 - val_acc: 0.6856
